In [11]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Lambda, Input
from utils import debug, compute_accuracy, eucl_dist_output_shape, euclidean_distance, contrastive_loss, generate_siamese_inputs
from tensorflow.keras.datasets import mnist, fashion_mnist
import tensorflow as tf
import numpy as np
from tensorflow.keras import backend as K

class Siamese_Net:
    def __init__(self, learning_rate, input_shape):
        self.learning_rate = learning_rate
        self.optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
        self.input_shape = input_shape
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(128, input_shape=self.input_shape, activation='relu'))
        # model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(128, activation='relu'))
        return model

    def train_one_step(self, x1, x2, y):
        with tf.GradientTape() as t:
            vector_a = self.model(x1)
            vector_b = self.model(x2)
            distance = Lambda(euclidean_distance,
                                output_shape=eucl_dist_output_shape)([vector_a, vector_b])
            loss = contrastive_loss(y_true=y, y_pred=distance)
            
            gradients = t.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss

tf.enable_eager_execution()
epoch_num = 40
batch_size = 128
siamese_net = Siamese_Net(learning_rate=1e-4, input_shape = (196,))

hfile = h5py.File('./data/n_segments_20_compactnes_1_sigma_1.h5', 'r')
X_train = np.array(hfile['feature'])
y_train = np.array(hfile['label'])
hfile.close()

train_dataset, data_size = generate_siamese_inputs(X_train, y_train, batch_size, shape=(-1, 196))
val_dataset, _ = generate_siamese_inputs(X_train, y_train, validation=True, shape=(-1, 196))



for epoch in range(epoch_num):
    num = 0
    for ((x1, x2), y) in train_dataset:
        loss = siamese_net.train_one_step(x1, x2, y)
        print("[INFO]epoch:{}, {}/{}, loss = {}".format(epoch, num, data_size//batch_size, loss))
        num+=1

for((val_X_1, val_X_2), val_label) in val_dataset:
    process_a = siamese_net.model.predict(val_X_1)
    process_b = siamese_net.model.predict(val_X_2)
    val_distance = Lambda(euclidean_distance,
                                    output_shape=eucl_dist_output_shape)([process_a, process_b])

    val_acc = compute_accuracy(y_true=val_label, y_pred=val_distance)
    print('[RESULT] Acc of validation dataset is {}'.format(val_acc))





siamese_net.model.summary()
siamese_net.model.save("./checkpoint/first_step/siamese_fashion_mnist_40.h5")



100%|██████████| 10/10 [00:00<00:00, 61.99it/s]


[INFO]epoch:0, 0/846, loss = 0.45756763219833374
[INFO]epoch:0, 1/846, loss = 0.5505630373954773
[INFO]epoch:0, 2/846, loss = 0.48857831954956055
[INFO]epoch:0, 3/846, loss = 0.4344829320907593
[INFO]epoch:0, 4/846, loss = 0.5118650794029236
[INFO]epoch:0, 5/846, loss = 0.5119585990905762
[INFO]epoch:0, 6/846, loss = 0.46514892578125
[INFO]epoch:0, 7/846, loss = 0.5349493026733398
[INFO]epoch:0, 8/846, loss = 0.5116903781890869
[INFO]epoch:0, 9/846, loss = 0.5194854140281677
[INFO]epoch:0, 10/846, loss = 0.5116884708404541
[INFO]epoch:0, 11/846, loss = 0.527370274066925
[INFO]epoch:0, 12/846, loss = 0.480561226606369
[INFO]epoch:0, 13/846, loss = 0.42617547512054443
[INFO]epoch:0, 14/846, loss = 0.48840197920799255
[INFO]epoch:0, 15/846, loss = 0.4884384572505951
[INFO]epoch:0, 16/846, loss = 0.44961029291152954
[INFO]epoch:0, 17/846, loss = 0.5037182569503784
[INFO]epoch:0, 18/846, loss = 0.449508935213089
[INFO]epoch:0, 19/846, loss = 0.5114763975143433
[INFO]epoch:0, 20/846, loss = 

KeyboardInterrupt: 